In [4]:
# Food Delivery Data Analysis – Hackathon Submission

import pandas as pd
import sqlite3
import os

# -------------------------------
# Check required files
# -------------------------------
required_files = ["orders.csv", "users.json", "restaurants.sql"]

for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"{file} not found. Please upload it.")

# -------------------------------
# Load Orders (CSV)
# -------------------------------
orders = pd.read_csv("orders.csv")

# -------------------------------
# Load Users (JSON)
# -------------------------------
users = pd.read_json("users.json")

# -------------------------------
# Load Restaurants (SQL)
# -------------------------------
conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql_query(
    "SELECT * FROM restaurants",
    conn
)

# -------------------------------
# Normalize column names
# -------------------------------
orders.columns = orders.columns.str.lower()
users.columns = users.columns.str.lower()
restaurants.columns = restaurants.columns.str.lower()

# -------------------------------
# Merge datasets (LEFT JOIN)
# -------------------------------
final_df = orders.merge(
    users,
    on="user_id",
    how="left"
)

final_df = final_df.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

# -------------------------------
# Date handling (NO WARNINGS)
# -------------------------------
if "order_date" in final_df.columns:
    final_df["order_date"] = pd.to_datetime(
        final_df["order_date"],
        dayfirst=True,
        errors="coerce"
    )
    final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

# -------------------------------
# Save final dataset
# -------------------------------
final_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

# =================================================
# Analysis
# =================================================

# Total Gold orders
gold_orders = (
    final_df[final_df["membership"] == "Gold"].shape[0]
    if "membership" in final_df.columns else 0
)

# Hyderabad revenue
hyderabad_revenue = (
    final_df.loc[final_df["city"] == "Hyderabad", "total_amount"].sum()
    if {"city", "total_amount"}.issubset(final_df.columns) else 0
)

# Distinct users
distinct_users = final_df["user_id"].nunique()

# Gold average order value
gold_avg_order_value = (
    final_df.loc[final_df["membership"] == "Gold", "total_amount"].mean()
    if {"membership", "total_amount"}.issubset(final_df.columns) else 0
)

# Orders with rating >= 4.5
high_rating_orders = (
    final_df.loc[final_df["rating"] >= 4.5].shape[0]
    if "rating" in final_df.columns else 0
)

# City with highest revenue among Gold members
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
    if {"membership", "city", "total_amount"}.issubset(final_df.columns) else "N/A"
)

# Highest revenue combination
top_revenue_combination = (
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
    .head(1)
    if {"membership", "cuisine", "total_amount"}.issubset(final_df.columns)
    else None
)

# -------------------------------
# Output
# -------------------------------
print("Final dataset created: final_food_delivery_dataset.csv\n")

print("===== KEY RESULTS =====")
print("Total Gold Orders:", gold_orders)
print("Hyderabad Revenue:", round(hyderabad_revenue, 2))
print("Distinct Users:", distinct_users)
print("Gold Average Order Value:", round(gold_avg_order_value, 2))
print("Orders with Rating >= 4.5:", high_rating_orders)
print("Top Gold Revenue City:", top_gold_city)

print("\nTop Revenue Combination:")
print(top_revenue_combination)


Final dataset created: final_food_delivery_dataset.csv

===== KEY RESULTS =====
Total Gold Orders: 4987
Hyderabad Revenue: 1889366.58
Distinct Users: 2883
Gold Average Order Value: 797.15
Orders with Rating >= 4.5: 3374
Top Gold Revenue City: Chennai

Top Revenue Combination:
membership  cuisine
Regular     Mexican    1072943.3
Name: total_amount, dtype: float64
